In [1]:
# %%
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
# exon-specific Δlogit
from helper import (
    load_and_align_for_delta_logit, pull_vectors_from_row,
    delta_logit_scores, curve_score_from_dlogit, hard_preds_from_dlogit,
    binary_metrics, find_contrastive_root, get_prediction_file,pull_pm1_vectors_from_row,
    pull_pm1_vectors_from_row_predfilter,
)
import time

# --- Detect and set CONTRASTIVE_ROOT environment variable ---
root_path = str(find_contrastive_root())
os.environ["CONTRASTIVE_ROOT"] = root_path
print(f"✅ CONTRASTIVE_ROOT set to: {root_path}")

# --- Root Paths ---
ROOT_RESULTS = f"{root_path}/files/results"
DATA_BASE = f"{root_path}/data/ASCOT"

# --- Ground Truth Binary Files ---
GT_HIGH = f"{DATA_BASE}/variable_cassette_exons_with_binary_labels_HIGH_TissueBinPsi.csv"
GT_LOW  = f"{DATA_BASE}/variable_cassette_exons_with_binary_labels_LOW_TissueBinPsi.csv"

# --- Output directory ---
OUT_DIR = f"{ROOT_RESULTS}/../classification_eval"
os.makedirs(OUT_DIR, exist_ok=True)


✅ CONTRASTIVE_ROOT set to: /gpfs/commons/home/atalukder/Contrastive_Learning


In [ ]:
def compute_classification_metrics_per_exon_logitdelta(
    gt_file: str,
    pred_file: str,
    model_name: str,
    expression_type: str  # "HIGH" or "LOW"
):
    merged, tissue_cols = load_and_align_for_delta_logit(
        gt_file=gt_file,
        pred_file=pred_file,
        require_cols=["logit_mean_psi"]
    )

    results = []
    for _, row in merged.iterrows():
        # 0/1 GT over tissues for this exon; PSI over tissues
        y_true, y_psi = pull_vectors_from_row(row, tissue_cols)
        if y_true.size == 0 or np.unique(y_true).size < 2:
            continue

        # Δlogit per tissue relative to this exon’s baseline
        muL = float(row["logit_mean_psi"])
        dlogit = delta_logit_scores(y_psi, muL)

        # Scores & hard predictions
        score_for_curve = curve_score_from_dlogit(dlogit, expression_type)
        y_pred = hard_preds_from_dlogit(dlogit, expression_type)

        m = binary_metrics(y_true, y_pred, score_for_curve)
        m.update({
            "model": model_name,
            "expression_type": expression_type,
            "exon_id": row["exon_id"],
        })
        results.append(m)

    df = pd.DataFrame(results)
    if df.empty:
        return df, f"⚠️ No valid exons found for {model_name} ({expression_type})\n"

    mean_metrics = df[["accuracy","precision","recall","f1","auroc","auprc"]].mean().round(4)
    summary_text = (
        f"\n📊 {model_name} ({expression_type} exons)\n"
        f"   n_exons   : {len(df)}\n" +
        "\n".join([f"   {k:<10}: {v:.4f}" for k,v in mean_metrics.items()]) + "\n"
    )
    # Print only the averages you care about
    avg_auprc = df["auprc"].mean(skipna=True)
    avg_auroc = df["auroc"].mean(skipna=True)
    print(f"📈 {model_name} [{expression_type}] — mean AUPRC = {avg_auprc:.4f}, mean AUROC = {avg_auroc:.4f}")

    return df, summary_text

In [12]:
def evaluate_tissue_binary_pm1_logitdelta_per_tissue_exonloop(
    gt_file: str,            # wide GT with {-1,0,+1} and 'logit_mean_psi'
    pred_file: str,          # predictions (PSI; % OK)
    model_name: str = "",
    filter_pred: bool = False,
    tissue_subset: list | None = None
) -> pd.DataFrame:
    """
    Binary tissue-specific evaluation:
      - Uses GT ∈ {-1,0,+1}; drops 0 and NaN; maps (-1 -> 0, +1 -> 1)
      - Score = Δlogit = logit(PSI_pred) − logit(mean_psi)
      - Hard label at 0: (Δlogit > 0) → 1
      - Aggregates per tissue across all exons in one exon-wise pass
    """
    merged, tissue_cols = load_and_align_for_delta_logit(
        gt_file=gt_file,
        pred_file=pred_file,
        require_cols=["logit_mean_psi"]
    )
    if tissue_subset is not None:
        tissue_cols = [t for t in tissue_cols if t in tissue_subset]

    # per-tissue accumulators
    acc = {t: {"y_true": [], "y_score": [], "y_pred": []} for t in tissue_cols}
    
    valid_exon = 0
    for _, row in merged.iterrows():
        # pull vectors for this exon (drop 0/NA, map -1/+1 -> 0/1)
        if not filter_pred:
            y_true_bin, y_psi, valid_mask = pull_pm1_vectors_from_row(row, tissue_cols)
        else:
            y_true_bin, y_psi, valid_mask = pull_pm1_vectors_from_row_predfilter(row, tissue_cols)
        if y_true_bin.size == 0:
            continue
        
        valid_exon+=1
        # per-position Δlogit relative to this exon’s baseline
        muL = float(row["logit_mean_psi"])
        dlogit = delta_logit_scores(y_psi, muL)  # higher ⇒ more +1

        # hard preds at Δlogit=0
        y_pred = (dlogit > 0).astype(int)

        # scatter back to tissues using the mask’s positions
        pos = 0
        for j, t in enumerate(tissue_cols):
            if valid_mask[j]:
                acc[t]["y_true"].append(int(y_true_bin[pos]))
                acc[t]["y_score"].append(float(dlogit[pos]))
                acc[t]["y_pred"].append(int(y_pred[pos]))
                pos += 1

    # compute metrics per tissue
    rows = []
    for t in tissue_cols:
        yt = np.asarray(acc[t]["y_true"], dtype=int)
        ys = np.asarray(acc[t]["y_score"], dtype=float)
        yp = np.asarray(acc[t]["y_pred"], dtype=int)
        if yt.size == 0 or np.unique(yt).size < 2 or not np.isfinite(ys).all():
            rows.append({"tissue": t, "model": model_name, "n": int(yt.size),
                         "accuracy": np.nan, "precision": np.nan, "recall": np.nan, "f1": np.nan,
                         "auroc": np.nan, "auprc": np.nan})
            continue
        m = binary_metrics(yt, yp, ys)
        m.update({"tissue": t, "model": model_name, "n": int(yt.size)})
        rows.append(m)

    return pd.DataFrame(rows).sort_values(["tissue","model"]).reset_index(drop=True)

def extract_valid_vectors(row, tissue_cols, gt_suffix="_gt", pred_suffix="_pred"):
    """Return (gt, pred, valid_mask) arrays for a merged exon row."""
    g = pd.to_numeric(row[[f"{t}{gt_suffix}" for t in tissue_cols]], errors="coerce").to_numpy(float)
    p = pd.to_numeric(row[[f"{t}{pred_suffix}" for t in tissue_cols]], errors="coerce").to_numpy(float)
    valid = np.isfinite(g) & np.isfinite(p)
    return g, p, valid

def evaluate_tissue_triclass_margin_per_tissue_exonloop(
    gt_file: str,
    pred_file: str,
    model_name: str = "",
    tissue_subset: list | None = None,
    margin_psi: float = 0.10,
    psi_bar_col: str = "mean_psi",
    use_logit_thresholds: bool = False,
    eps: float = 1e-6
) -> pd.DataFrame:
    """
    Three-class tissue-specific evaluation aligned to GT rule:
      GT: psi_pred > psi_bar + m -> +1; psi_pred < psi_bar - m -> -1; else 0.
      If use_logit_thresholds = True, we classify via Δlogit using per-exon τ_pos/τ_neg
      derived from the same ±m band (exactly equivalent to the PSI rule).
    Metrics: accuracy, macro-F1, weighted-F1 per tissue.
    """

    import numpy as np
    import pandas as pd
    from sklearn.metrics import accuracy_score, f1_score

    # helper functions reused
    def sigmoid(z): return 1.0 / (1.0 + np.exp(-z))
    def logit(p):   return np.log(p) - np.log1p(-p)

    # --- load data ---
    merged, tissue_cols = load_and_align_for_delta_logit(
        gt_file=gt_file,
        pred_file=pred_file,
        require_cols=["logit_mean_psi"]
    )
    if psi_bar_col not in merged.columns:
        merged[psi_bar_col] = sigmoid(merged["logit_mean_psi"].astype(float))
    if tissue_subset is not None:
        tissue_cols = [t for t in tissue_cols if t in tissue_subset]

    acc = {t: {"y_true": [], "y_pred": []} for t in tissue_cols}

    # --- iterate over exons ---
    for _, row in merged.iterrows():
        # use existing helper for vector extraction
        g, p, valid = extract_valid_vectors(row, tissue_cols)
        if not np.any(valid):
            continue

        psi_bar = float(row[psi_bar_col])
        psi_bar_clip = np.clip(psi_bar, eps, 1 - eps)
        muL = float(row["logit_mean_psi"])

        if use_logit_thresholds:
            # convert PSI thresholds to Δlogit thresholds (exact)
            psi_hi = np.clip(psi_bar + margin_psi, eps, 1 - eps)
            psi_lo = np.clip(psi_bar - margin_psi, eps, 1 - eps)
            tau_pos = logit(psi_hi) - muL
            tau_neg = logit(psi_lo) - muL
            dlogit = delta_logit_scores(p[valid], muL)
            # dlogit = compute_dlogit(p[valid], muL)

            y_pred = np.where(
                dlogit > tau_pos,  1,
                np.where(dlogit < tau_neg, -1, 0)
            )
        else:
            # direct PSI-based classification
            pv = p[valid]
            y_pred = np.where(
                pv > psi_bar + margin_psi,  1,
                np.where(pv < psi_bar - margin_psi, -1, 0)
            )

        # --- scatter to tissues ---
        pos = 0
        for j, t in enumerate(tissue_cols):
            if valid[j]:
                acc[t]["y_true"].append(int(g[j]))
                acc[t]["y_pred"].append(int(y_pred[pos]))
                pos += 1

    # --- compute metrics per tissue ---
    rows = []
    for t in tissue_cols:
        yt = np.asarray(acc[t]["y_true"], dtype=int)
        yp = np.asarray(acc[t]["y_pred"], dtype=int)
        if yt.size == 0 or np.unique(yt).size < 2:
            rows.append({
                "tissue": t, "model": model_name, "n": int(yt.size),
                "accuracy": np.nan, "macro_f1": np.nan, "weighted_f1": np.nan
            })
            continue

        rows.append({
            "tissue": t,
            "model": model_name,
            "n": int(yt.size),
            "accuracy": accuracy_score(yt, yp),
            "macro_f1": f1_score(yt, yp, average="macro"),
            "weighted_f1": f1_score(yt, yp, average="weighted")
        })

    return pd.DataFrame(rows).sort_values(["tissue", "model"]).reset_index(drop=True)


In [ ]:
def evaluate_exonSpecific_classification(result_file_name, model_user_name_norun):
    pred_file = get_prediction_file(ROOT_RESULTS, result_file_name)
    print(f"📂 Found predictions for {model_user_name_norun} → {pred_file}")

    df_high, txt_high = compute_classification_metrics_per_exon_logitdelta(
        GT_HIGH, pred_file, model_user_name_norun, "HIGH"
    )
    df_low,  txt_low  = compute_classification_metrics_per_exon_logitdelta(
        GT_LOW,  pred_file, model_user_name_norun, "LOW"
    )

    combined = pd.concat([df_high, df_low], ignore_index=True)
    out_csv = f"{OUT_DIR}/{model_user_name_norun}_classification_metrics_logitdelta.csv"
    combined.to_csv(out_csv, index=False)
    print(f"✅ Saved classification metrics → {out_csv}")
    return combined, txt_high + txt_low


In [ ]:
# # --- Paths you likely already have ---
# GT_CLASS3_WIDE = f"{DATA_BASE}/variable_cassette_exons_with_binary_labels_ExonBinPsi.csv"  # {-1,0,+1} wide

# def evaluate_tissueSpecific_classification(result_file_name: str, model_user_name_norun: str):
#     """
#     Tissue-specific binary eval using GT {-1,+1} only (drops 0/NA).
#     Score = Δlogit; metrics aggregated per tissue across exons.
#     Requires GT_CLASS3_WIDE to contain columns: exon_id, logit_mean_psi, and tissue columns.
#     """
#     # 1) prediction file
#     pred_file = get_prediction_file(ROOT_RESULTS, result_file_name)
#     print(f"📂 Found predictions for {model_user_name_norun} → {pred_file}")

#     # 2) Run tissue-specific eval (alignment, scaling, masking handled inside)
#     per_tissue_df = evaluate_tissue_binary_pm1_logitdelta_per_tissue_exonloop(
#         gt_file=GT_CLASS3_WIDE,
#         pred_file=pred_file,
#         model_name=model_user_name_norun,
#         tissue_subset=None  # or pass a list to restrict/ordering
#     )

#     # 3) Save + summary text
#     out_csv = f"{OUT_DIR}/{model_user_name_norun}_tissue_metrics_logitdelta.csv"
#     per_tissue_df.to_csv(out_csv, index=False)
#     print(f"✅ Saved tissue metrics → {out_csv}")

#     n_tissues = per_tissue_df["tissue"].nunique()
#     macro = per_tissue_df[["accuracy","precision","recall","f1","auroc","auprc"]].mean().round(4)
#     summary = (
#         f"\n🧪 {model_user_name_norun} (tissue-specific)\n"
#         f"   n_tissues : {n_tissues}\n" +
#         "\n".join([f"   {k:<10}: {v:.4f}" for k,v in macro.items()]) + "\n"
#     )

#     # 4) Print only the averages you asked for
#     avg_auprc = per_tissue_df["auprc"].mean(skipna=True)
#     avg_auroc = per_tissue_df["auroc"].mean(skipna=True)
#     print(f"📈 {model_user_name_norun}: mean AUPRC = {avg_auprc:.4f}, mean AUROC = {avg_auroc:.4f}")

#     return per_tissue_df, summary


In [4]:
# # --- Paths you likely already have ---
GT_CLASS3_WIDE = f"{DATA_BASE}/variable_cassette_exons_with_binary_labels_ExonBinPsi.csv"  # {-1,0,+1} wide

def evaluate_tissueSpecific_classification(result_file_name: str, model_user_name_norun: str, filter_pred: bool = False):
    """
    Tissue-specific classification evaluation.
    1️⃣ Binary: GT {-1,+1} only (drops 0/NA) via Δlogit threshold at 0.
    2️⃣ Tri-class: GT {-1,0,+1} using ±margin_psi rule (Δlogit or PSI thresholds).

    Both produce per-tissue CSVs and print summary stats.
    Returns (combined_df, summary_text)
    """
    import numpy as np
    import pandas as pd

    # 1️⃣ Locate prediction file
    pred_file = get_prediction_file(ROOT_RESULTS, result_file_name)
    print(f"📂 Found predictions for {model_user_name_norun} → {pred_file}")

    # --------------------------------------------------------------------------
    # 2️⃣ Run Binary tissue-specific evaluation
    per_tissue_bin = evaluate_tissue_binary_pm1_logitdelta_per_tissue_exonloop(
        gt_file=GT_CLASS3_WIDE,
        pred_file=pred_file,
        model_name=model_user_name_norun,
        tissue_subset=None,
        filter_pred=filter_pred
    )

    out_bin_csv = f"{OUT_DIR}/{model_user_name_norun}_tissue_metrics_logitdelta_binary.csv"
    per_tissue_bin.to_csv(out_bin_csv, index=False)
    print(f"✅ Saved binary tissue metrics → {out_bin_csv}")

    # --- Binary summary
    n_tissues = per_tissue_bin["tissue"].nunique()
    macro_bin = per_tissue_bin[["accuracy","precision","recall","f1","auprc", "auroc"]].mean(skipna=True).round(4)
    summary_bin = (
        f"\n🧪 {model_user_name_norun} (Binary)\n"
        f"   n_tissues : {n_tissues}\n" +
        "\n".join([f"   {k:<10}: {v:.4f}" for k,v in macro_bin.items()]) + "\n"
    )
    print(summary_bin)

    # --------------------------------------------------------------------------
    # 3️⃣ Run Tri-class tissue-specific evaluation
    per_tissue_tri = evaluate_tissue_triclass_margin_per_tissue_exonloop(
        gt_file=GT_CLASS3_WIDE,
        pred_file=pred_file,
        model_name=model_user_name_norun,
        tissue_subset=None,          # restrict to certain tissues if needed
        margin_psi=0.10,             # matches GT ±0.1 rule
        use_logit_thresholds=True    # set False to use PSI space thresholds
    )

    out_tri_csv = f"{OUT_DIR}/{model_user_name_norun}_tissue_metrics_triclass.csv"
    per_tissue_tri.to_csv(out_tri_csv, index=False)
    print(f"✅ Saved tri-class tissue metrics → {out_tri_csv}")

    # --- Tri-class summary
    n_tissues_tri = per_tissue_tri["tissue"].nunique()
    macro_tri = per_tissue_tri[["accuracy","macro_f1","weighted_f1"]].mean(skipna=True).round(4)
    summary_tri = (
        f"\n🧬 {model_user_name_norun} (Tri-class)\n"
        f"   n_tissues : {n_tissues_tri}\n" +
        "\n".join([f"   {k:<10}: {v:.4f}" for k,v in macro_tri.items()]) + "\n"
    )
    print(summary_tri)

    # --------------------------------------------------------------------------
    # 4️⃣ Combined summary
    print(f"📊 {model_user_name_norun}: "
          f"Binary F1={macro_bin['f1']:.4f}, "
          f"Tri-class macroF1={macro_tri['macro_f1']:.4f}")

    combined_summary = summary_bin + "\n" + summary_tri

    # For backward compatibility with your existing code
    combined_df = pd.concat([
        per_tissue_bin.assign(eval_type="binary"),
        per_tissue_tri.assign(eval_type="triclass")
    ], ignore_index=True)

    return combined_df, combined_summary


In [5]:
def plot_classification_summary(results_df, metric="auprc", out_name=None):
    plt.figure(figsize=(10, 5))
    sns.boxplot(
        data=results_df,
        x="expression_type",
        y=metric,
        hue="model",
        showfliers=False
    )
    plt.ylabel(metric.upper())
    plt.title(f"Per-exon {metric.upper()} distribution across models (HIGH/LOW exons)")
    plt.legend(loc="upper right", fontsize=8)
    plt.tight_layout()
    if out_name:
        out_path = f"{OUT_DIR}/{out_name}.png"
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        print(f"📊 Saved figure → {out_path}")
    plt.show()

In [14]:
time_stamp = time.strftime("%Y_%m_%d__%H_%M_%S", time.localtime())
SUMMARY_TXT = f"{OUT_DIR}/ExonSpecific_classification_summary_{time_stamp}.txt"
summary_lines = ["===== MODEL CLASSIFICATION SUMMARY ====="]

# --- Model 1 ---
result_file_name1 = "exprmnt_2025_11_03__23_37_19" # EMPRAIPsi_ASCOT_CLSwpd_200bp_5aug_5p3pCut_2025_11_03__23_37_19
model1_user_name_norun = 'ASCOT_CLSwpd_200bp_5aug_5p3pCut'
# --- Model 2 ---
result_file_name2 =  "exprmnt_2025_11_03__23_40_11" # EMPRAIPsi_ASCOT_CLSwpd_200bp_10aug_5p3pCut_2025_11_03__23_40_11
model2_user_name_norun = 'ASCOT_CLSwpd_200bp_10aug_5p3pCut'

# --- SOTA ---
sota_result_dir = "mtsplice_originalTFweight_results/intron_300bp_results"
sota_model_name_norun = "MTSplice_original_SOTA"
sota_pred_file = f"{ROOT_RESULTS}/{sota_result_dir}/variable_all_tissues_predicted_psi.tsv"
print(f"📂 Using SOTA file → {sota_pred_file}")


📂 Using SOTA file → /gpfs/commons/home/atalukder/Contrastive_Learning/files/results/mtsplice_originalTFweight_results/intron_300bp_results/variable_all_tissues_predicted_psi.tsv


In [ ]:
# # DO NOT ERASE (AT)

# # Exon specifidic classification evaluation

# # --- Model 1 ---
# df_model1, txt_model1 = evaluate_exonSpecific_classification(result_file_name1, model1_user_name_norun)
# summary_lines += [txt_model1]

# # --- Model 2 ---'
# df_model2, txt_model2 = evaluate_exonSpecific_classification(result_file_name2, model2_user_name_norun)
# summary_lines += [txt_model2]

# # --- SOTA ---
# df_sota_high, txt_sota_high = compute_classification_metrics_per_exon_logitdelta(GT_HIGH, sota_pred_file, sota_model_name_norun, "HIGH")
# df_sota_low, txt_sota_low  = compute_classification_metrics_per_exon_logitdelta(GT_LOW,  sota_pred_file, sota_model_name_norun, "LOW")
# summary_lines += [txt_sota_high, txt_sota_low]

# df_sota = pd.concat([df_sota_high, df_sota_low], ignore_index=True)
# df_sota.to_csv(f"{OUT_DIR}/{sota_model_name_norun}_classification_metrics.csv", index=False)

# # with open(SUMMARY_TXT, "w") as f:
# #     f.write("\n".join(summary_lines))

# combined_all = pd.concat([df_sota, df_model1, df_model2], ignore_index=True)
# combined_all.to_csv(f"{OUT_DIR}/combined_classification_summary.csv", index=False)
# print(f"✅ Combined summary saved → {OUT_DIR}/combined_classification_summary.csv")

# plot_classification_summary(combined_all, metric="auprc", out_name="AUPRC_SOTA_vs_models")
# plot_classification_summary(combined_all, metric="auroc", out_name="AUROC_SOTA_vs_models")


In [16]:
# tissue specific classification evaluation

# --- Model 1 ---
df_model1, txt_model1 = evaluate_tissueSpecific_classification(result_file_name1, model1_user_name_norun, filter_pred=True)
summary_lines += [txt_model1]

# --- Model 2 ---'
df_model2, txt_model2 = evaluate_tissueSpecific_classification(result_file_name2, model2_user_name_norun, filter_pred=True)
summary_lines += [txt_model2]

# # --- SOTA ---

# 1️⃣ Binary (existing)
df_sota_bin = evaluate_tissue_binary_pm1_logitdelta_per_tissue_exonloop(
    gt_file=GT_CLASS3_WIDE,
    pred_file=sota_pred_file,
    model_name=sota_model_name_norun,
    filter_pred=True
)

auroc_bin = df_sota_bin["auroc"].mean(skipna=True)
auprc_bin = df_sota_bin["auprc"].mean(skipna=True)
print(f"{sota_model_name_norun}  [Binary]  AUPRC={auprc_bin:.4f} AUROC={auroc_bin:.4f}")

metric_cols_bin = ["accuracy","precision","recall","f1","auroc","auprc"]
macro_bin = df_sota_bin[metric_cols_bin].mean(skipna=True).round(4)
txt_sota_bin = (
    f"\n🧪 {sota_model_name_norun} (Binary)\n"
    f"   n_tissues : {df_sota_bin['tissue'].nunique()}\n" +
    "\n".join(f"   {k:<10}: {v:.4f}" for k, v in macro_bin.items()) + "\n"
)
summary_lines += [txt_sota_bin]

# Optionally save
df_sota_bin.to_csv(f"{OUT_DIR}/{sota_model_name_norun}_tissue_metrics_binary.csv", index=False)
print(f"✅ Saved binary SOTA results → {OUT_DIR}/{sota_model_name_norun}_tissue_metrics_binary.csv")

# ------------------------------------------------------------------------------

# 2️⃣ Tri-class (new)
df_sota_tri = evaluate_tissue_triclass_margin_per_tissue_exonloop(
    gt_file=GT_CLASS3_WIDE,
    pred_file=sota_pred_file,
    model_name=sota_model_name_norun,
    tissue_subset=None,
    margin_psi=0.10,
    use_logit_thresholds=True  # same Δlogit thresholds rule
)

macro_tri = df_sota_tri[["accuracy","macro_f1","weighted_f1"]].mean(skipna=True).round(4)
# Build formatted summary string (matches Binary print style)
txt_sota_tri = (
    f"\n🧬 {sota_model_name_norun} (Tri-class)\n"
    f"   n_tissues : {df_sota_tri['tissue'].nunique()}\n" +
    "\n".join([
        f"   {k:<10}: {v:.4f}" 
        for k, v in macro_tri.items()
    ]) + "\n"
)

# Append to summary list and print
summary_lines += [txt_sota_tri]
print(txt_sota_tri)

# Save CSV
out_tri_csv = f"{OUT_DIR}/{sota_model_name_norun}_tissue_metrics_triclass.csv"
df_sota_tri.to_csv(out_tri_csv, index=False)
print(f"✅ Saved tri-class SOTA results → {out_tri_csv}")


# df_sota.to_csv(f"{OUT_DIR}/{sota_model_name_norun}_tissue_metrics_logitdelta.csv", index=False)



# df_sota.to_csv(f"{OUT_DIR}/{sota_model_name_norun}_classification_metrics.csv", index=False)

# with open(SUMMARY_TXT, "w") as f:
#     f.write("\n".join(summary_lines))

📂 Found predictions for ASCOT_CLSwpd_200bp_5aug_5p3pCut → /gpfs/commons/home/atalukder/Contrastive_Learning/files/results/exprmnt_2025_11_03__23_37_19/ensemble_evaluation_from_valdiation/test_set_evaluation/tsplice_final_predictions_all_tissues.tsv
✅ Saved binary tissue metrics → /gpfs/commons/home/atalukder/Contrastive_Learning/files/results/../classification_eval/ASCOT_CLSwpd_200bp_5aug_5p3pCut_tissue_metrics_logitdelta_binary.csv

🧪 ASCOT_CLSwpd_200bp_5aug_5p3pCut (Binary)
   n_tissues : 56
   accuracy  : 0.7969
   precision : 0.2868
   recall    : 0.3478
   f1        : 0.3143
   auprc     : 0.6398
   auroc     : 0.6610

✅ Saved tri-class tissue metrics → /gpfs/commons/home/atalukder/Contrastive_Learning/files/results/../classification_eval/ASCOT_CLSwpd_200bp_5aug_5p3pCut_tissue_metrics_triclass.csv

🧬 ASCOT_CLSwpd_200bp_5aug_5p3pCut (Tri-class)
   n_tissues : 56
   accuracy  : 0.1582
   macro_f1  : 0.1219
   weighted_f1: 0.0684

📊 ASCOT_CLSwpd_200bp_5aug_5p3pCut: Binary F1=0.3143, 

In [ ]:



def plot_tissue_classification_box(results_df, metric="auprc", out_name=None,
                                       model_order=None, palette=None, show_points=True):
    """
    Boxplot of per-tissue metrics across models (no pre-aggregation).
    Expects columns: ['tissue', 'model', <metric>].
    """
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Keep only finite metric values; no groupby needed
    df = results_df.replace([np.inf, -np.inf], np.nan).dropna(subset=[metric, "model"])

    plt.figure(figsize=(10, 5))
    ax = sns.boxplot(
        data=df,
        x="model",
        y=metric,
        order=model_order,
         hue="model",
        showfliers=False
    )

    if show_points:
        sns.stripplot(
            data=df,
            x="model",
            y=metric,
            order=model_order,
            dodge=False,
            jitter=0.12,
            size=3,
            alpha=0.65,
            color="k"  # keep legend clean
        )

    if metric.lower() in {"auprc", "auroc"}:
        plt.ylim(0, 1)

    plt.ylabel(metric.upper())
    plt.title(f"Per-tissue {metric.upper()} distribution across models")
    plt.xlabel("model")
    plt.tight_layout()

    if out_name:
        out_path = f"{OUT_DIR}/{out_name}.png"
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        print(f"📊 Saved figure → {out_path}")

    plt.show()



In [ ]:
combined_all = pd.concat([df_sota, df_model1, df_model2], ignore_index=True)
combined_all.to_csv(f"{OUT_DIR}/combined_tissue_classification_summary.csv", index=False)
print(f"✅ Combined summary saved → {OUT_DIR}/combined_tissue_classification_summary.csv")

# Use the tissue plotter (not the exon HIGH/LOW plotter)
# plot_tissue_classification_summary(combined_all, metric="auprc", out_name="AUPRC_tissue_SOTA_vs_models")
# plot_tissue_classification_summary(combined_all, metric="auroc", out_name="AUROC_tissue_SOTA_vs_models")

plot_tissue_classification_box(combined_all, metric="auprc", out_name="AUPRC_tissue_box_SOTA_vs_models")
plot_tissue_classification_box(combined_all, metric="auroc", out_name="AUROC_tissue_box_SOTA_vs_models")
